In [4]:
import os
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from pathlib import Path
import preprocessing


current_folder = Path()
dataset_folder = current_folder / "dataset"
images_folder = dataset_folder / "images"
models_folder = current_folder / "models"
logs_folder = current_folder / "logs"

X_train, X_test, y_train, y_test = preprocessing.get_dataset()
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

INFO: modified dataset already created


In [5]:
np.unique(y_train, return_counts=True)

(array(['crosswalk', 'speedlimit', 'stop', 'trafficlight'], dtype='<U12'),
 array([118, 543,  71,  94]))

In [6]:
from imblearn.over_sampling import SMOTE

X_flat = np.reshape(X_train, (X_train.shape[0], int(np.product(X_train.shape) / X_train.shape[0])))

sm = SMOTE(n_jobs=-1, random_state=42)
X_train_os, y_train_os = sm.fit_resample(X_flat, y_train)

X_train_os_rs = np.reshape(X_train_os, tuple([X_train_os.shape[0]]) + X_train.shape[1:])

X_train = X_train_os_rs
y_train = y_train_os

In [7]:
np.unique(y_train, return_counts=True)

(array(['crosswalk', 'speedlimit', 'stop', 'trafficlight'], dtype='<U12'),
 array([543, 543, 543, 543]))

In [8]:
# One hot encoding
label_enc = sklearn.preprocessing.LabelEncoder()
y_train = label_enc.fit_transform(y_train)
y_test = label_enc.transform(y_test)
one_hot = sklearn.preprocessing.OneHotEncoder(sparse=False)
y_train = one_hot.fit_transform(y_train.reshape(-1, 1))
y_test = one_hot.transform(y_test.reshape(-1, 1))

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.25, random_state=54
)

In [9]:
print(np.sum(y_train, axis=0) / len(y_train))
print(np.sum(y_test, axis=0) / len(y_train))

[0.25046041 0.24248005 0.25659914 0.25046041]
[0.01780233 0.09146716 0.00736648 0.01043585]


In [10]:
import tensorflow as tf
from tensorflow import keras
from functools import partial

# Keras model
DefaultConv2D = partial(
    keras.layers.Conv2D, kernel_size=3, activation="relu", padding="SAME"
)

model = keras.Sequential(
    [
        DefaultConv2D(
            filters=64, kernel_size=7, input_shape=list(X_train[0].shape)
        ),  # was 28, 28, 1
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=128),
        DefaultConv2D(filters=128),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=256),
        DefaultConv2D(filters=256),
        keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation="relu"),
        keras.layers.Dropout(0.2),  # lower less regularization
        keras.layers.Dense(units=64, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(units=4, activation="softmax"),
    ]
)

model.compile(
    loss="categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"],
)

# loss
# sparse_categorical_crossentropy used for sparse labels (target class index)
# categorial_cross_entropy would yield a one-hot vector (only one positive label)
# mean_squared_error for regression

tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=logs_folder,
    histogram_freq=1,
)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=False,
)

print("Start training")
history = model.fit(
    X_train,
    y_train,
    epochs=500,
    validation_data=(X_valid, y_valid),
    callbacks=[tensorboard_cb, early_stopping_cb],
)

# Save model
os.makedirs(models_folder, exist_ok=True)
model.save(models_folder / "model1.h5")

2021-12-24 12:32:41.092944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-24 12:32:41.148241: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-24 12:32:41.148634: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-24 12:32:41.150293: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Start training


2021-12-24 12:32:42.213981: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 195480000 exceeds 10% of free system memory.
2021-12-24 12:32:42.355022: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 195480000 exceeds 10% of free system memory.


Epoch 1/500


2021-12-24 12:32:44.977451: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300
2021-12-24 12:32:47.140058: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


51/51 [==============================] - ETA: 0s - loss: 1.3195 - accuracy: 0.3757

2021-12-24 12:32:49.903219: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 65160000 exceeds 10% of free system memory.
2021-12-24 12:32:49.970590: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 65160000 exceeds 10% of free system memory.
2021-12-24 12:32:50.897229: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 35389440 exceeds 10% of free system memory.


51/51 [==============================] - 10s 88ms/step - loss: 1.3195 - accuracy: 0.3757 - val_loss: 1.2756 - val_accuracy: 0.5985
Epoch 2/500
51/51 [==============================] - 3s 55ms/step - loss: 0.9253 - accuracy: 0.6538 - val_loss: 0.6973 - val_accuracy: 0.7864
Epoch 3/500
51/51 [==============================] - 2s 43ms/step - loss: 0.4034 - accuracy: 0.8686 - val_loss: 0.6021 - val_accuracy: 0.8379
Epoch 4/500
51/51 [==============================] - 2s 44ms/step - loss: 0.3051 - accuracy: 0.9128 - val_loss: 0.0925 - val_accuracy: 0.9853
Epoch 5/500
51/51 [==============================] - 2s 41ms/step - loss: 0.2335 - accuracy: 0.9380 - val_loss: 0.1154 - val_accuracy: 0.9779
Epoch 6/500
51/51 [==============================] - 2s 41ms/step - loss: 0.1409 - accuracy: 0.9656 - val_loss: 0.0883 - val_accuracy: 0.9853
Epoch 7/500
51/51 [==============================] - 2s 44ms/step - loss: 0.0787 - accuracy: 0.9828 - val_loss: 0.0506 - val_accuracy: 0.9890
Epoch 8/500
51/51

In [11]:
# Evaluate per class
y_pred = model.predict(X_test)
Y_pred = np.argmax(y_pred, axis=1)  # one-hot to index
Y_test = np.argmax(y_test, axis=1)
print(
    sklearn.metrics.classification_report(
        Y_test, Y_pred, target_names=label_enc.classes_
    )
)

# Evaluate general
test_results = model.evaluate(X_test, y_test)  # loss and metrics
print(f"Test Data - Loss: {test_results[0]:.3f}, Metrics: {test_results[1:]}")

              precision    recall  f1-score   support

   crosswalk       0.90      0.97      0.93        29
  speedlimit       0.99      0.98      0.99       149
        stop       1.00      1.00      1.00        12
trafficlight       0.88      0.88      0.88        17

    accuracy                           0.97       207
   macro avg       0.94      0.96      0.95       207
weighted avg       0.97      0.97      0.97       207

7/7 [==============================] - 0s 12ms/step - loss: 0.1214 - accuracy: 0.9710
Test Data - Loss: 0.121, Metrics: [0.9710144996643066]
